### Parameters and targets
Set up some basic parameters and targets

In [ ]:
from typing import List
import numpy as np
import pandas as pd
pd.options.plotting.backend = 'plotly'
from scipy.optimize import minimize, shgo
import nevergrad as ng

from emu_renewal.distributions import get_gamma_params_from_mean_sd, get_gamma_densities_from_params
from emu_renewal.process import get_interp_vals_over_model_time
from emu_renewal.renew import renew_basic
from emu_renewal.outputs import Outputs, plot_output_fit

In [ ]:
def model_func(gen_time_mean: float, gen_time_sd: float, process_req: List[float], pop: int, seed: int, n_times: int) -> tuple:
    """The common features of the model.
    Get the generation time distribution from the user request,
    linearly interpolate the non-mechanistic process,
    exponentiate the result and run the renewal process.
    """
    gen_time_densities = get_gamma_densities_from_params(gen_time_mean, gen_time_sd, n_times)
    process_vals = get_interp_vals_over_model_time(process_req, n_times)
    process_vals_exp = np.exp(np.array(process_vals))
    model_result = renew_basic(gen_time_densities, process_vals_exp, pop, seed, n_times)
    return model_result, process_vals_exp

def calib_func(parameters: List[float], pop: int, seed: int, n_times: int, targets: dict) -> float:
    """Additionally include the generation time parameters as parameters,
    least squares loss function.
    """
    gen_time_mean, gen_time_sd, *process_req = parameters
    incidence = model_func(gen_time_mean, gen_time_sd, process_req, pop, seed, n_times)[0][0]
    return sum([(incidence[t] - d) ** 2 for t, d in targets.items()])

In [ ]:
n_times = 40
infectious_seed = 1.0
population = 100.0

### Calibrate to outputs produced by model
Check that the optimisation algorithm can recover
the parameters used in generating some arbitrary data.
This actually worked better (closer fit to data)
when using the raw process values rather than their logs.

In [ ]:
model_times = pd.Series(range(n_times))
test_process = [1.8, 2.5, 1.6, 0.7]
test_data, _ = model_func(5.5, 1.8, np.log(test_process), population, infectious_seed, n_times)
test_vals = pd.Series(test_data.incidence, index=model_times)

In [ ]:
# Global optimisation with shgo - need to capture arguments through closure due to bug in optimisation function
# as per comment at https://stackoverflow.com/questions/72794609/scipy-issue-passing-arguments-to-optimize-shgo-function
param_bounds = [[0.1, 10.0]] + [[0.1, 4.0]] + [[np.log(1.0), np.log(5.0)]] * 4
global_result = shgo(lambda x, p=population, s=infectious_seed, t=n_times, d=test_vals: calib_func(x, p, s, t, d), param_bounds)
model_result, process_vals = model_func(global_result.x[0], global_result.x[1], global_result.x[2:], population, infectious_seed, n_times)
optimised, suscept, r_t = model_result
print(test_process)
print(np.exp(np.array(global_result.x[2:])))
plot_output_fit(test_vals, model_result, process_vals, n_times)

### Nevergrad optimisation with synthetic data
Use NGOpt to optimise directly to the outputs from the model with known parameters.

In [ ]:
def obj_func(parameters):
    return calib_func(parameters, pop=1e2, seed=1.0, n_times=n_times, targets=test_vals)
optimizer = ng.optimizers.NGOpt(parametrization=6, budget=10000)
ngopt_result = optimizer.minimize(obj_func)
ngopt_output, ngopt_process = model_func(ngopt_result.value[0], ngopt_result.value[1], ngopt_result.value[2:], 1e2, 1.0, len(test_vals))
print(test_process)
print(np.exp(ngopt_result.value[2:]))
plot_output_fit(test_vals, ngopt_output, ngopt_process, n_times)

Doesn't work quite as well with `TwoPointsDE`.

In [ ]:
optimizer = ng.optimizers.TwoPointsDE(parametrization=6, budget=10000)
tpde_result = optimizer.minimize(obj_func)
tpde_output, tpde_process = model_func(tpde_result.value[0], tpde_result.value[1], tpde_result.value[2:], 1e2, 1.0, len(test_vals))
print(test_process)
print(np.exp(tpde_result.value[2:]))
plot_output_fit(test_vals, tpde_output, tpde_process, n_times)

### Nevergrad optimisation with jittered synthetic data
NGOpt against synthetic data with jitter applied.

In [ ]:
spread = 0.2
jitter = pd.Series(np.random.normal(scale=spread, size=n_times) + 1.0)
jittered_vals = test_vals * jitter
def obj_func(parameters):
    return calib_func(parameters, pop=1e2, seed=1.0, n_times=n_times, targets=jittered_vals)
optimizer = ng.optimizers.NGOpt(parametrization=6, budget=10000)
jitter_result = optimizer.minimize(obj_func)
jitter_output, process_vals = model_func(jitter_result.value[0], jitter_result.value[1], jitter_result.value[2:], 1e2, 1.0, len(test_vals))
print(test_process)
print(np.exp(jitter_result.value[2:]))
plot_output_fit(jittered_vals, jitter_output, process_vals, n_times)